In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.8 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=6f5db6665aac9ae4c6375115f1540b492ba4815813ee257e435b019a3a57f7d1
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
import numpy as np

from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-mpnet-base-v2')

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

Collect all segments and summaries :

In [ ]:
fileNames = ["Verifiable", "Soft", "Non-verifiable"]

def openSegments(fileName):
  # Open collection of segments:
  f = open("drive/MyDrive/data/"+fileName+"_segments.txt", "r")

  data = []
  segmentText = ""
  for line in f:
      try:
          if '>>>' in line:
              # memorize previous segment:
              if segmentText != "":
                  data.append(segmentText.strip())
                  segmentText = ""
          else:
              segmentText += line

      except ValueError:
          print('Invalid input:',line)

  # Memorize the last segment:
  data.append(segmentText.strip())

  f.close()

  return data

def openSummaries(fileName,i):
  # Open collection of segments:
  f = open("drive/MyDrive/data/"+fileName+"_summaries_"+str(i)+".txt", "r")

  data = []
  segmenttext = ""
  for line in f:
      try:
          if '>>>' in line:
              # memorize previous segment:
              if segmenttext != "":
                  data.append(segmenttext.strip())
                  segmenttext = ""
          else:
              segmenttext += line

      except ValueError:
          print('Invalid input:',line)

  # Memorize the last segment:
  data.append(segmenttext.strip())

  # Extract abstracts:
  abstracts = []
  for segment in data:
      if segment != 0:
          line = segment.split('>> Résumé :\n\n')[1]
          summaryText = line.split('> ')[1].strip()
          if not 'Error!' in summaryText:
              abstracts.append(summaryText)

  f.close()

  return abstracts

Load and process data:

In [ ]:
for f_name in fileNames:
    segments = openSegments(f_name)
    resumes_1 = openSummaries(f_name,0)
    resumes_2 = openSummaries(f_name,1)
    resumes_3 = openSummaries(f_name,2)
    resumes_4 = openSummaries(f_name,3)

    # Compute similarities and save the results:
    f_out = open("drive/MyDrive/data/"+f_name+"_bert_similarities.csv","w")

    f_out.write("1,2,3,4\n")

    for i in range(len(segments)):
      # Compute similarities:
      query_embedding = model.encode(segments[i].replace('\n',''))
      passage_embedding = model.encode([resumes_1[i], resumes_2[i], resumes_3[i], resumes_4[i]])
      scores = util.dot_score(query_embedding, passage_embedding)

      # 2-step evaluation:
      ranks = np.argsort(np.argsort([float(i) for i in scores[0]]))
      ranks = [4-ranks[j] if float(scores[0][j])>=0.50 else 0 for j in range(len(ranks))]

      # Save the results:
      for j in range(3):
        f_out.write(str(ranks[j])+",")
      f_out.write(str(ranks[3])+"\n")

    f_out.close()